In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
import random


match_info = []
table = []
for i in range(21925, 22305):
  if 22014<i<22275:
    continue
  url = f'https://understat.com/match/{i}'
  res = requests.get(url)
  soup = BeautifulSoup(res.content, "lxml")
  scripts = soup.find_all('script')

  import json

  string_with_json_obj = ''

  for el in scripts:
    if 'shotsData' in el.text:
      string_with_json_obj = el.text.strip()

  ind_start = string_with_json_obj.index("('")+2
  ind_end = string_with_json_obj.index("')")
  json_data = string_with_json_obj[ind_start:ind_end]

  json_data = json_data.encode('utf8').decode('unicode_escape')

  data = json.loads(json_data)


  f_h_xG, f_a_xG = 0, 0
  for shoot in data['h']:
    f_id = i
    f_match = f'''{shoot['h_team']} - {shoot['a_team']}. {shoot['date'][:16]}, {shoot['h_goals']}:{shoot['a_goals']}'''
    f_team = f'''{shoot['h_team']}'''
    f_minute = int(shoot['minute'])
    f_xG = float(shoot['xG'])
    f_player = f'''{shoot['player']}'''
    f_situation = f'''{shoot['situation']}'''
    f_result = f'''{shoot['result']}'''
    f_shot_type = f'''{shoot['shotType']}'''
    f_assistant = f'''{shoot['player_assisted']}'''
    f_last_action = f'''{shoot['lastAction']}'''
    f_opposite_team = f'''{shoot['a_team']}'''
    f_date = f'''{shoot['date']}'''

    table.append([f_id, f_match, f_team, f_minute, f_xG, f_player, f_situation, f_result, f_shot_type, f_assistant, f_last_action, f_opposite_team, f_date])

    f_h_xG += f_xG

  for shoot in data['a']:
    f_id = i
    f_match = f'''{shoot['h_team']} - {shoot['a_team']}. {shoot['date'][:16]}, {shoot['h_goals']}:{shoot['a_goals']}'''
    f_team = f'''{shoot['a_team']}'''
    f_minute = int(shoot['minute'])
    f_xG = float(shoot['xG'])
    f_player = f'''{shoot['player']}'''
    f_situation = f'''{shoot['situation']}'''
    f_result = f'''{shoot['result']}'''
    f_shot_type = f'''{shoot['shotType']}'''
    f_assistant = f'''{shoot['player_assisted']}'''
    f_last_action = f'''{shoot['lastAction']}'''
    f_opposite_team = f'''{shoot['h_team']}'''
    f_date = f'''{shoot['date']}'''

    table.append([f_id, f_match, f_team, f_minute, f_xG, f_player, f_situation, f_result, f_shot_type, f_assistant, f_last_action, f_opposite_team, f_date])

    f_a_xG += f_xG

  match_info.append([i, shoot['h_team'], shoot['a_team'], shoot['date'][:16], int(shoot['h_goals']), int(shoot['a_goals']), f_h_xG, f_a_xG, len(data['h']), len(data['a'])])

  time.sleep(random.randint(13, 17))

print(*match_info, sep='\n')

In [ ]:
import pandas as pd

header_table = ['id', 'match', 'team', 'minute', 'xG', 'player', 'situation', 'result', 'shot_type', 'assistant', 'last_action', 'opposite_team', 'date']
df = pd.DataFrame(table, columns=header_table)
df.to_csv('epl23_24shots.csv', sep=';', encoding='utf-16')

header_match = ['id', 'team_home', 'team_away', 'date', 'h_goals', 'a_goals', 'h_xG', 'a_xG', 'h_shots', 'a_shots']
df = pd.DataFrame(match_info, columns=header_match)
df.to_csv('epl23_24matches.csv', sep=';', encoding='utf-16')